In [1]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
main_df = pd.read_csv('movies.csv')
df = main_df.copy()
print(df.shape)
df.head()

(4803, 24)


,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
#relevant features for recommendation

selected_feature = ['genres','keywords','tagline','cast','director','title']
selected_feature

['genres', 'keywords', 'tagline', 'cast', 'director', 'title']

In [4]:
new_df = df[selected_feature]
new_df

,genres,keywords,tagline,cast,director,title
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron,Avatar
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski,Pirates of the Caribbean: At World's End
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes,Spectre
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan,The Dark Knight Rises
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton,John Carter
...,...,...,...,...,...,...
4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,"He didn't come looking for trouble, but troubl...",Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez,El Mariachi
4799,Comedy Romance,NaN,A newlywed couple's honeymoon is upended by th...,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns,Newlyweds
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,NaN,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith,"Signed, Sealed, Delivered"
4801,NaN,NaN,A New Yorker in Shanghai,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia,Shanghai Calling


In [5]:
new_df.isna().sum()

genres       28
keywords    412
tagline     844
cast         43
director     30
title         0
dtype: int64

In [6]:
for i in new_df.columns:
    new_df[i]=new_df[i].fillna('')

C:\Users\Abhishek\AppData\Local\Temp\ipykernel_71076\3739387675.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[i]=new_df[i].fillna('')


In [7]:
new_df.head()

,genres,keywords,tagline,cast,director,title
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron,Avatar
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski,Pirates of the Caribbean: At World's End
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes,Spectre
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan,The Dark Knight Rises
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton,John Carter


In [8]:
new_df.isna().sum()

genres      0
keywords    0
tagline     0
cast        0
director    0
title       0
dtype: int64

In [9]:
combined_feat = new_df['genres']+' '+new_df['keywords']+' '+new_df['tagline']+' '+new_df['cast']+' '+new_df['director']+' '+new_df['title']

In [10]:
combined_feat[0]

'Action Adventure Fantasy Science Fiction culture clash future space war space colony society Enter the World of Pandora. Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez James Cameron Avatar'

In [11]:
#converting into feature vector

vect = TfidfVectorizer()
vect

TfidfVectorizer()

In [12]:
feature_vect = vect.fit_transform(combined_feat)
# print(feature_vect)

In [13]:
#using cosine similarity to get the similarity score

similarity = cosine_similarity(feature_vect)
print(similarity)

[[1.         0.07294698 0.03533251 ... 0.         0.         0.        ]
 [0.07294698 1.         0.02792771 ... 0.04419983 0.         0.        ]
 [0.03533251 0.02792771 1.         ... 0.         0.04636139 0.        ]
 ...
 [0.         0.04419983 0.         ... 1.         0.         0.05551043]
 [0.         0.         0.04636139 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.05551043 0.         1.        ]]


In [14]:
similarity.shape

(4803, 4803)

In [15]:
movie_name = input('Enter the name of Movie: ')

In [16]:
#Creating a list with names of all the movies in the dataset

list_of_movies = df['title'].to_list()

In [17]:
# print(list_of_movies)

In [18]:
#Finding the closest match of the movie name given by the user

best_match = difflib.get_close_matches(movie_name, list_of_movies)
print(best_match)

['Batman', 'Batman', 'Catwoman']


In [19]:
close_match = best_match[0]
close_match

'Batman'

In [20]:
#finding the index of movie based on title

index_of_movie = df[df['title']==close_match]['index'].values[0]
index_of_movie 

1359

In [21]:
similarity_score = list(enumerate(similarity[index_of_movie]))

In [22]:
# print(similarity_score)

In [23]:
#sorting the movies based on thier similarity score

sorted_similarity_score = sorted(similarity_score, key = lambda x : x[1] ,reverse=True)

In [24]:
sorted_similarity_score[:10]

[(1359, 1.0),
 (428, 0.4443306267625309),
 (210, 0.28943211113215106),
 (119, 0.23005137142418236),
 (3, 0.1882934516088265),
 (65, 0.1761708480005976),
 (299, 0.14584607116516127),
 (1512, 0.1367918072744968),
 (813, 0.13670426148197926),
 (4267, 0.13488837306856133)]

In [25]:
for a,b in sorted_similarity_score[:10]:
    print(a,b)

1359 1.0
428 0.4443306267625309
210 0.28943211113215106
119 0.23005137142418236
3 0.1882934516088265
65 0.1761708480005976
299 0.14584607116516127
1512 0.1367918072744968
813 0.13670426148197926
4267 0.13488837306856133


In [26]:
#find the title based on similarty

movies = []
for i,name in sorted_similarity_score[:10]:
    name = df[df['index']==i]['title'].values[0]
    movies.append(name)

In [27]:
print(movies)

['Batman', 'Batman Returns', 'Batman & Robin', 'Batman Begins', 'The Dark Knight Rises', 'The Dark Knight', 'Batman Forever', 'A History of Violence', 'Superman', 'Batman']


Pred Function

In [28]:
# movie_name = input('Enter the name of Movie: ')
def recommend_movies(movie_name):
    # movie_name = input('Enter the name of Movie: ')

    list_of_movies = df['title'].to_list()

    best_match = difflib.get_close_matches(movie_name, list_of_movies)

    close_match = best_match[0]

    index_of_movie = df[df['title']==close_match]['index'].values[0]

    similarity_score = list(enumerate(similarity[index_of_movie]))

    sorted_similarity_score = sorted(similarity_score, key = lambda x : x[1] ,reverse=True)

    movies_list = []
    for i,name in sorted_similarity_score[:10]:
        name = df[df['index']==i]['title'].values[0]
        movies_list.append(name)
    
    return movies_list


In [29]:
recommend_movies('Pirates of the Caribbean')

["Pirates of the Caribbean: At World's End",
 'Pirates of the Caribbean: The Curse of the Black Pearl',
 "Pirates of the Caribbean: Dead Man's Chest",
 'Anna and the King',
 'Seeking a Friend for the End of the World',
 'Cast Away',
 'Pirates of the Caribbean: On Stranger Tides',
 'The Lord of the Rings: The Return of the King',
 "The World's End",
 'This Is the End']

In [30]:
recommend_movies('Iron man')

['Iron Man',
 'Iron Man 2',
 'Iron Man 3',
 'The Avengers',
 'Avengers: Age of Ultron',
 'Ant-Man',
 'Captain America: Civil War',
 'Captain America: The Winter Soldier',
 'X-Men',
 'Made']

In [31]:
recommend_movies('Batman')

['Batman',
 'Batman Returns',
 'Batman & Robin',
 'Batman Begins',
 'The Dark Knight Rises',
 'The Dark Knight',
 'Batman Forever',
 'A History of Violence',
 'Superman',
 'Batman']